In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Model class
class ZurcherModel:
    def __init__(self, n=175, max_mileage=450, RC=11.7257, c=2.45569, beta=0.9999, p=[0.0937, 0.4475, 0.4459, 0.0127]):
        self.n = n
        self.max_mileage = max_mileage
        self.RC = RC
        self.c = c
        self.beta = beta
        self.p = np.array(p)
        self.grid = np.arange(n)
        self.cost = 0.001 * c * self.grid
        self.state_transition()

    def state_transition(self):
        p = np.append(self.p, 1 - np.sum(self.p))
        P1 = np.zeros((self.n, self.n))
        for i in range(self.n):
            if i <= self.n - len(p):
                P1[i, i:i + len(p)] = p
            else:
                P1[i, i:] = p[:self.n - i]
                P1[i, -1] = 1.0 - np.sum(P1[i, :-1])
        P2 = np.zeros((self.n, self.n))
        for i in range(self.n):
            P2[i, :len(p)] = p
        self.P1 = P1
        self.P2 = P2

    def bellman(self, ev0):
        value_keep = -self.cost + self.beta * (self.P1 @ ev0)
        value_replace = -self.RC - self.cost[0] + self.beta * ev0[0]
        maxV = np.maximum(value_keep, value_replace)
        logsum = maxV + np.log(np.exp(value_keep - maxV) + np.exp(value_replace - maxV))
        ev1 = self.P1 @ logsum
        pk = 1 / (1 + np.exp(value_replace - value_keep))
        return ev1, pk

# NFXP Solver class
class NFXP:
    def __init__(self, model, tol=1e-10, max_iter=50):
        self.model = model
        self.tol = tol
        self.max_iter = max_iter

    def solve(self):
        ev = np.zeros(self.model.n)
        for i in range(self.max_iter):
            ev1, pk = self.model.bellman(ev)
            if np.max(np.abs(ev1 - ev)) < self.tol:
                return ev1, pk
            ev = ev1
        raise ValueError("NFXP did not converge")

# Log-likelihood function
def log_likelihood(theta, model, data):
    RC, c = theta
    model.RC, model.c = RC, c
    model.cost = 0.001 * c * model.grid
    solver = NFXP(model)
    ev, pk = solver.solve()
    log_lik = np.log(pk[data.x] * (1 - data.d) + (1 - pk[data.x]) * data.d)
    return -np.mean(log_lik)

# Load and preprocess data
def load_data(file_path):
    data = pd.read_csv(file_path)
    data = data.dropna()
    data['x'] = (data['x'] * 175 / (450 * 1000)).astype(int)
    return data

# Main function
def main():
    # Load data
    file_path = "busdata1234.csv"
    data = load_data(file_path)

    # Initialize model
    model = ZurcherModel()

    # Initial guess for parameters
    theta0 = [12, 2.5]

    # Estimate parameters
    res = minimize(log_likelihood, theta0, args=(model, data), method='BFGS')

    print("Estimated parameters:", res.x)
    print("Log-likelihood value:", -res.fun)

if __name__ == "__main__":
    main()
